In [157]:
import findspark
findspark.init()  # Инициализация Spark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

# Создаём SparkSession
spark = SparkSession.builder \
    .appName("Jupyter-Spark") \
    .master("local[*]") \
    .getOrCreate()

# Проверяем, что Spark работает
spark

In [147]:
# Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)
df1 = spark.read.option("header", "true").csv("covid-data.csv")
df1.select('iso_code', 'location', 'population', 'total_cases').filter(df1.date <= '2021-03-31').groupBy('iso_code', 'location').\
    agg(F.sum("total_cases").alias("total_cases"), F.min("population").alias("population")).\
    select('iso_code', 'location', F.col('total_cases')/F.col('population')).sort(F.col('(total_cases / population)').desc()).show(15)

+--------+-------------+--------------------------+
|iso_code|     location|(total_cases / population)|
+--------+-------------+--------------------------+
|     AND|      Andorra|        20.809357406328868|
|     SMR|   San Marino|         16.37008662855796|
|     MNE|   Montenegro|        14.777818113498348|
|     QAT|        Qatar|        14.164645304158887|
|     BHR|      Bahrain|        13.656087302235624|
|     CZE|      Czechia|        13.287636770703322|
|     LUX|   Luxembourg|         13.13894622158038|
|     USA|United States|         12.53664414351345|
|     PAN|       Panama|        12.028483570843207|
|     ISR|       Israel|        11.996087708440177|
|     BEL|      Belgium|        10.437872661182217|
|     SVN|     Slovenia|        10.320871485936047|
|     ARM|      Armenia|        10.115108020493825|
|     ESP|        Spain|         9.465290406759026|
|     PRT|     Portugal|         9.087371246422988|
+--------+-------------+--------------------------+
only showing

In [153]:
#Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
#(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)
df2 = spark.read.option("header", "true").csv("covid-data.csv")
df2.select('date', 'location', 'new_cases').\
    filter((F.col('date') >= '2021-03-25') & (F.col('date') <= '2021-03-31') & (F.col('continent') != '')).\
    groupBy('date', 'location').\
    agg(F.sum("new_cases").cast('int').alias("new_cases")).\
    sort(F.col('new_cases').desc()).show(10)

+----------+-------------+---------+
|      date|     location|new_cases|
+----------+-------------+---------+
|2021-03-25|       Brazil|   100158|
|2021-03-31|       Brazil|    90638|
|2021-03-27|       Brazil|    85948|
|2021-03-30|       Brazil|    84494|
|2021-03-26|       Brazil|    84245|
|2021-03-26|United States|    77321|
|2021-03-31|        India|    72330|
|2021-03-29|United States|    69429|
|2021-03-28|        India|    68020|
|2021-03-25|United States|    67465|
+----------+-------------+---------+
only showing top 10 rows



In [169]:
#Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. 
#(например: в россии вчера было 9150 , сегодня 8763, итог: -387) 
#(в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)
from pyspark.sql import Window
from pyspark.sql.functions import col, lag 
df2 = spark.read.option("header", "true").csv("covid-data.csv")
w = Window.partitionBy().orderBy("date")
df2.select('date', 'location', 'new_cases').\
    filter((F.col('date') >= '2021-03-25') & (F.col('date') <= '2021-03-31') & (F.col('location') == 'Russia')).\
    withColumn('prv_new_cases', lag('new_cases').over(w)).\
    withColumn('delta', F.col('new_cases') - F.col('prv_new_cases')).show()

+----------+--------+---------+-------------+------+
|      date|location|new_cases|prv_new_cases| delta|
+----------+--------+---------+-------------+------+
|2021-03-25|  Russia|   9128.0|         NULL|  NULL|
|2021-03-26|  Russia|   9073.0|       9128.0| -55.0|
|2021-03-27|  Russia|   8783.0|       9073.0|-290.0|
|2021-03-28|  Russia|   8979.0|       8783.0| 196.0|
|2021-03-29|  Russia|   8589.0|       8979.0|-390.0|
|2021-03-30|  Russia|   8162.0|       8589.0|-427.0|
|2021-03-31|  Russia|   8156.0|       8162.0|  -6.0|
+----------+--------+---------+-------------+------+

